# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
data.select('crew').show()

+----+
|crew|
+----+
|3.55|
|3.55|
| 6.7|
|19.1|
|10.0|
| 9.2|
| 9.2|
| 9.2|
| 9.2|
|11.5|
|11.6|
| 6.6|
| 9.2|
| 9.2|
| 9.3|
|11.6|
|10.3|
| 9.2|
| 9.3|
| 9.2|
+----+
only showing top 20 rows



In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [16]:
from pyspark.ml.feature import StringIndexer 
cruise_new =StringIndexer(inputCol='Cruise_line',outputCol='cruise_num') 
trans =cruise_new.fit(data).transform(data) 

In [17]:
assembler = VectorAssembler(
    inputCols=["cruise_num", "Age", "Tonnage", 
               "passengers","length", "cabins", "passenger_density"],
    outputCol="features")

In [19]:
output = assembler.transform(trans)

In [20]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[16.0,6.0,30.2769...|
|[16.0,6.0,30.2769...|
|[1.0,26.0,47.262,...|
|[1.0,11.0,110.0,2...|
|[1.0,17.0,101.353...|
|[1.0,22.0,70.367,...|
|[1.0,15.0,70.367,...|
|[1.0,23.0,70.367,...|
|[1.0,19.0,70.367,...|
|[1.0,6.0,110.2389...|
|[1.0,10.0,110.0,2...|
|[1.0,28.0,46.052,...|
|[1.0,18.0,70.367,...|
|[1.0,17.0,70.367,...|
|[1.0,11.0,86.0,21...|
|[1.0,8.0,110.0,29...|
|[1.0,9.0,88.5,21....|
|[1.0,15.0,70.367,...|
|[1.0,12.0,88.5,21...|
|[1.0,20.0,70.367,...|
+--------------------+
only showing top 20 rows



In [21]:
final_data = output.select("features",'crew')

In [22]:
data_train,data_test = final_data.randomSplit([0.7,0.3])

In [23]:
data_train.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              115|
|   mean|7.738869565217398|
| stddev|3.709898037379169|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [24]:
data_test.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                43|
|   mean| 7.942093023255814|
| stddev|2.9145362762241183|
|    min|              1.46|
|    max|              13.6|
+-------+------------------+



In [25]:
lin_reg = LinearRegression(labelCol='crew')

In [27]:
lin_reg_model = lin_reg.fit(data_train,)

In [28]:
print("Coefficients: {} Intercept: {}".format(lin_reg_model.coefficients,lin_reg_model.intercept))

Coefficients: [0.04559222557965382,-0.018544451316896127,0.0033670388208513306,-0.12884529542150305,0.43160849002098406,0.8614750489659598,-0.0005897557795453196] Intercept: -1.0732097091551012


In [29]:
predict = lin_reg_model.evaluate(data_test)

In [31]:
predict.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.4577971318533365|
| -0.7061736243817816|
|  -1.114645318675505|
| -1.1675564160417125|
| -0.6519387149252616|
|-0.46506298823547443|
| -0.6554577422855292|
| -0.6412293758688428|
|-0.30266038318596244|
|   0.492774833853483|
| -0.8017037207351017|
|  0.5298637364872771|
|  0.5919304672456267|
| 0.46706264104816064|
| 0.08560430217547754|
| 0.25670177448672815|
|-0.12516228502540994|
| -1.3695321822426738|
|  -0.517240745312975|
|-0.20269384344025543|
+--------------------+
only showing top 20 rows



In [32]:
unlabeled_data = data_test.select('features')

In [33]:
predict_final = lin_reg_model.transform(unlabeled_data)

In [34]:
predict_final.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[0.0,6.0,158.0,43...|14.057797131853336|
|[0.0,9.0,90.09,25...| 9.396173624381781|
|[0.0,11.0,138.0,3...|12.964645318675505|
|[0.0,13.0,138.0,3...|12.927556416041712|
|[0.0,16.0,78.491,...| 8.301938714925262|
|[0.0,17.0,70.0,20...| 7.665062988235475|
|[1.0,8.0,110.0,29...|12.255457742285529|
|[1.0,9.0,110.0,29...|12.241229375868842|
|[1.0,11.0,86.0,21...| 9.602660383185963|
|[1.0,17.0,70.367,...| 8.707225166146516|
|[1.0,17.0,101.353...|10.801703720735102|
|[1.0,19.0,70.367,...| 8.670136263512722|
|[1.0,23.0,70.367,...| 8.608069532754373|
|[1.0,26.0,47.262,...|6.2329373589518395|
|[2.0,6.0,113.0,37...|11.914395697824522|
|[2.0,14.0,30.2769...| 3.473298225513272|
|[2.0,22.0,69.845,...|  7.08516228502541|
|[3.0,5.0,86.0,21....| 9.369532182242674|
|[3.0,13.0,61.0,13...| 6.517240745312975|
|[3.0,20.0,55.451,...| 5.772693843440256|
+--------------------+------------

In [35]:
print("RMSE: {}".format(predict.rootMeanSquaredError))
print("MSE: {}".format(predict.meanSquaredError))

RMSE: 0.910675594754274
MSE: 0.8293300388810505
